In [22]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import os
from PIL import Image

# Define the noise functions
def noisy(noise_typ, image):
    if noise_typ == "gauss":
        row, col, ch = image.shape
        mean = 0
        var = 0.01
        sigma = var**0.5
        gauss = np.random.normal(mean, sigma, (row, col, ch))
        gauss = gauss.reshape(row, col, ch)
        noisy = image + gauss
        return np.clip(noisy, 0, 1)  # Clip values to ensure they stay within [0, 1]
    elif noise_typ == "saltpepper":
        row, col = image.shape[:2]  # Only use height and width (ignore channels)
        s_vs_p = 0.5
        amount = 0.004
        out = np.copy(image)
        
        # Salt mode
        num_salt = np.ceil(amount * row * col * s_vs_p)
        coords = [np.random.randint(0, i, int(num_salt)) for i in (row, col)]
        out[coords[0], coords[1]] = 1  # Set salt pixels to 1
        
        # Pepper mode
        num_pepper = np.ceil(amount * row * col * (1. - s_vs_p))
        coords = [np.random.randint(0, i, int(num_pepper)) for i in (row, col)]
        out[coords[0], coords[1]] = 0  # Set pepper pixels to 0
        
        return out
    elif noise_typ == "poisson":
        vals = len(np.unique(image))
        vals = 2 ** np.ceil(np.log2(vals))
        noisy = np.random.poisson(image * vals) / float(vals)
        return noisy
    elif noise_typ == "speckle":
        row, col, ch = image.shape
        gauss = np.random.randn(row, col, ch)
        gauss = gauss.reshape(row, col, ch)
        noisy = image + image * gauss
        return noisy

# Load the FashionMNIST dataset
transform = transforms.Compose([transforms.ToTensor()])
trainset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=1, shuffle=False)

# Create a directory to save noisy images
output_dir = './mixed_noisy_fashionmnist'
os.makedirs(output_dir, exist_ok=True)

# Create subdirectories for each class
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
for i, class_name in enumerate(class_names):
    os.makedirs(os.path.join(output_dir, class_name), exist_ok=True)

# Add noise to the dataset (half Gaussian, half Salt & Pepper)
dataset_size = len(trainset)
half_size = dataset_size // 2

for idx, (image, label) in enumerate(trainloader):
    # Convert image to numpy array
    img = image.squeeze().numpy()  # Remove batch dimension and convert to numpy
    img = np.expand_dims(img, axis=-1)  # Add channel dimension
    
    # Apply Gaussian noise to the first half
    if idx < half_size:
        noisy_img = noisy("gauss", img)
    # Apply Salt & Pepper noise to the second half
    else:
        noisy_img = noisy("saltpepper", img)
    
    noisy_img = np.squeeze(noisy_img)  # Remove channel dimension
    
    # Convert back to PIL image
    noisy_img = (noisy_img * 255).astype(np.uint8)  # Scale to 0-255
    noisy_img = Image.fromarray(noisy_img, mode='L')  # Convert to PIL image
    
    # Save the noisy image
    class_name = class_names[label]
    save_path = os.path.join(output_dir, class_name, f'noisy_{idx}.png')
    noisy_img.save(save_path)

    if idx % 1000 == 0:
        print(f"Processed {idx + 1} images")

print("Mixed noisy dataset saved successfully!")

Processed 1 images
Processed 1001 images
Processed 2001 images
Processed 3001 images
Processed 4001 images
Processed 5001 images
Processed 6001 images
Processed 7001 images
Processed 8001 images
Processed 9001 images
Processed 10001 images
Processed 11001 images
Processed 12001 images
Processed 13001 images
Processed 14001 images
Processed 15001 images
Processed 16001 images
Processed 17001 images
Processed 18001 images
Processed 19001 images
Processed 20001 images
Processed 21001 images
Processed 22001 images
Processed 23001 images
Processed 24001 images
Processed 25001 images
Processed 26001 images
Processed 27001 images
Processed 28001 images
Processed 29001 images
Processed 30001 images
Processed 31001 images
Processed 32001 images
Processed 33001 images
Processed 34001 images
Processed 35001 images
Processed 36001 images
Processed 37001 images
Processed 38001 images
Processed 39001 images
Processed 40001 images
Processed 41001 images
Processed 42001 images
Processed 43001 images
P